In [1]:
import sys
sys.path.append('../../..')
from Library.lossSRM import LossAbs
from Library.CompoundPoisson import CompoundPoisson
import numpy as np
import scipy.stats as st
from scipy.optimize import minimize

In [2]:
class MCLossFunction(LossAbs):
    def __init__(self, X, alpha, c = None):
        self.__alpha = alpha
        super(MCLossFunction, self).__init__(X, c)
        
    def shortfall_risk(self, m = None):
        m = self._check_argument(m)
        #This substract from the col i of X, m_i
        #X is a matrix of dim columns and N rows where N is the sample's length
        x_minus_m = np.subtract(self.X, m)
        #axis = 1 means we sum the elements of each row
        mean_sum = np.mean(np.sum(x_minus_m, axis = 1))
        pos_part = np.maximum(x_minus_m, 0)
        mean_sum_2 = np.mean(np.sum(pos_part ** 2, axis = 1))
        cross_term = 0.
        for i in range(self.dim):
            for j in range(i + 1, self.dim):
                cross_term += np.mean(np.multiply(pos_part[:, i], pos_part[:, j]))
        return mean_sum + 0.5 * mean_sum_2 + self.__alpha * cross_term
    
    def shortfall_risk_jac(self, m):
        m = self._check_argument(m)
        x_minus_m = np.subtract(self.X, m)
        pos_part = np.maximum(x_minus_m, 0) 
        pos_part_mean = np.mean(pos_part, axis = 0)
        cross_jac = []
        for i in range(self.dim):
            temp = 0.
            indic_i = np.sign(pos_part[:, i])
            for k in range(self.dim):
                if k != i:
                    temp += np.mean(np.multiply(pos_part[:, k], indic_i))
            cross_jac.append(temp)
        return pos_part_mean + 1. + cross_jac
        

In [10]:
#Don't execute this another time, this was done to calculate the time execution of the CompounPoisson Class
#Case: rho = -0.9, alpha = 1

#M is the length of the sample
M = 100000
rho = [[1, -0.9],[-0.9, 1]]
lam = [2, 3]
T = 1
mu = 0
sigma = 1
obj = CompoundPoisson(T, lam, rho,  M, mu, sigma)
%timeit distr = obj.compound_poisson()

c = 1.
alpha  = 1.
loss = MCLossFunction(distr, alpha, c)
maxiter = 3500

init = np.ones(loss.dim)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})
res = minimize(loss.objective, init, jac = loss.objective_jac, constraints = cons, method='SLSQP',options={'maxiter': maxiter, 'disp': True})
print(res)

5min 33s ± 629 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.31364912213492874
            Iterations: 7
            Function evaluations: 12
            Gradient evaluations: 7
     fun: 0.31364912213492874
     jac: array([1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 12
     nit: 7
    njev: 7
  status: 0
 success: True
       x: array([0.00134615, 0.31230297])


In [17]:
#Case: rho = -0.9, alpha = 1

#M is the length of the sample
M = 100000
rho = [[1, -0.9],[-0.9, 1]]
lam = [2, 3]
T = 1
mu = 0
sigma = 1
obj = CompoundPoisson(T, lam, rho,  M, mu, sigma)
distr = obj.compound_poisson()

c = 1.
alpha  = 1.
loss = MCLossFunction(distr, alpha, c)
maxiter = 3500

init = np.ones(loss.dim)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})
res = minimize(loss.objective, init, jac = loss.objective_jac, constraints = cons, method='SLSQP',options={'maxiter': maxiter, 'disp': True})
print(res)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.30673694239814875
            Iterations: 6
            Function evaluations: 6
            Gradient evaluations: 6
     fun: 0.30673694239814875
     jac: array([1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 6
     nit: 6
    njev: 6
  status: 0
 success: True
       x: array([0.00652744, 0.30020951])


In [18]:
#Case: rho = -0.5, alpha = 1

#M is the length of the sample
M = 100000
rho = [[1, -0.5],[-0.5, 1]]
lam = [2, 3]
T = 1
mu = 0
sigma = 1
obj = CompoundPoisson(T, lam, rho,  M, mu, sigma)
distr = obj.compound_poisson()

c = 1.
alpha  = 1.
loss = MCLossFunction(distr, alpha, c)
maxiter = 3500

init = np.ones(loss.dim)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})
res = minimize(loss.objective, init, jac = loss.objective_jac, constraints = cons, method='SLSQP',options={'maxiter': maxiter, 'disp': True})
print(res)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.31099937325766225
            Iterations: 7
            Function evaluations: 7
            Gradient evaluations: 7
     fun: 0.31099937325766225
     jac: array([1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 7
     nit: 7
    njev: 7
  status: 0
 success: True
       x: array([0.01670205, 0.29429733])


In [19]:
#Case: rho = 0, alpha = 1
M = 100000
rho = [[1, 0],[0, 1]]
lam = [2, 3]
T = 1
mu = 0
sigma = 1
obj = CompoundPoisson(T, lam, rho,  M, mu, sigma)
distr = obj.compound_poisson()

c = 1.
alpha  = 1.
loss = MCLossFunction(distr, alpha, c)
maxiter = 3500

init = np.ones(loss.dim)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})
res = minimize(loss.objective, init, jac = loss.objective_jac, constraints = cons, method='SLSQP',options={'maxiter': maxiter, 'disp': True})
print(res)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.3222971808317298
            Iterations: 6
            Function evaluations: 6
            Gradient evaluations: 6
     fun: 0.3222971808317298
     jac: array([1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 6
     nit: 6
    njev: 6
  status: 0
 success: True
       x: array([0.02704583, 0.29525135])


In [21]:
#Case: rho = 0.5, alpha = 1
M = 100000
rho = [[1, 0.5],[0.5, 1]]
lam = [2, 3]
T = 1
mu = 0
sigma = 1
obj = CompoundPoisson(T, lam, rho,  M, mu, sigma)
distr = obj.compound_poisson()

c = 1.
alpha  = 1.
loss = MCLossFunction(distr, alpha, c)
maxiter = 3500

init = np.ones(loss.dim)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})
res = minimize(loss.objective, init, jac = loss.objective_jac, constraints = cons, method='SLSQP',options={'maxiter': maxiter, 'disp': True})
print(res)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.3372269890866782
            Iterations: 6
            Function evaluations: 6
            Gradient evaluations: 6
     fun: 0.3372269890866782
     jac: array([1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 6
     nit: 6
    njev: 6
  status: 0
 success: True
       x: array([0.0305482 , 0.30667879])


In [22]:
#Case: rho = 0.9, alpha = 1
M = 100000
rho = [[1, 0.9],[0.9, 1]]
lam = [2, 3]
T = 1
mu = 0
sigma = 1
obj = CompoundPoisson(T, lam, rho,  M, mu, sigma)
distr = obj.compound_poisson()

c = 1.
alpha  = 1.
loss = MCLossFunction(distr, alpha, c)
maxiter = 3500

init = np.ones(loss.dim)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})
res = minimize(loss.objective, init, jac = loss.objective_jac, constraints = cons, method='SLSQP',options={'maxiter': maxiter, 'disp': True})
print(res)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.3485454502481555
            Iterations: 6
            Function evaluations: 6
            Gradient evaluations: 6
     fun: 0.3485454502481555
     jac: array([1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 6
     nit: 6
    njev: 6
  status: 0
 success: True
       x: array([0.04655685, 0.3019886 ])


In [5]:
#case of identically distriubted variables with rho = -0.9 and alpha = 1

M = 100000
rho = [[1, -0.9],[-0.9, 1]]
lam = [3, 3]
T = 1
mu = 0
sigma = 1
obj = CompoundPoisson(T, lam, rho,  M, mu, sigma)
distr = obj.compound_poisson()

c = 1.
alpha  = 1.
loss = MCLossFunction(distr, alpha, c)
maxiter = 3500

init = np.ones(loss.dim)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})
res = minimize(loss.objective, init, jac = loss.objective_jac, constraints = cons, method='SLSQP',options={'maxiter': maxiter, 'disp': True})
print(res)


Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.46230396298896925
            Iterations: 5
            Function evaluations: 6
            Gradient evaluations: 5
     fun: 0.46230396298896925
     jac: array([1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 6
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([0.23051307, 0.23179089])


In [6]:
#case of identically distriubted variables with rho = -0.5 and alpha = 1

M = 100000
rho = [[1, -0.5],[-0.5, 1]]
lam = [3, 3]
T = 1
mu = 0
sigma = 1
obj = CompoundPoisson(T, lam, rho,  M, mu, sigma)
distr = obj.compound_poisson()

c = 1.
alpha  = 1.
loss = MCLossFunction(distr, alpha, c)
maxiter = 3500

init = np.ones(loss.dim)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})
res = minimize(loss.objective, init, jac = loss.objective_jac, constraints = cons, method='SLSQP',options={'maxiter': maxiter, 'disp': True})
print(res)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.4755438232388367
            Iterations: 5
            Function evaluations: 6
            Gradient evaluations: 5
     fun: 0.4755438232388367
     jac: array([1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 6
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([0.23787804, 0.23766578])


In [7]:
#case of identically distriubted variables with rho = 0 and alpha = 1

M = 100000
rho = [[1, 0],[0, 1]]
lam = [3, 3]
T = 1
mu = 0
sigma = 1
obj = CompoundPoisson(T, lam, rho,  M, mu, sigma)
distr = obj.compound_poisson()

c = 1.
alpha  = 1.
loss = MCLossFunction(distr, alpha, c)
maxiter = 3500

init = np.ones(loss.dim)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})
res = minimize(loss.objective, init, jac = loss.objective_jac, constraints = cons, method='SLSQP',options={'maxiter': maxiter, 'disp': True})
print(res)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.47931523358349054
            Iterations: 5
            Function evaluations: 6
            Gradient evaluations: 5
     fun: 0.47931523358349054
     jac: array([1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 6
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([0.24013089, 0.23918435])


In [10]:
#case of identically distriubted variables with rho = 0.5 and alpha = 1
M = 100000
rho = [[1, 0.5],[0.5, 1]]
lam = [3, 3]
T = 1
mu = 0
sigma = 1
obj = CompoundPoisson(T, lam, rho,  M, mu, sigma)
distr = obj.compound_poisson()

c = 1.
alpha  = 1.
loss = MCLossFunction(distr, alpha, c)
maxiter = 3500

init = np.ones(loss.dim)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})
res = minimize(loss.objective, init, jac = loss.objective_jac, constraints = cons, method='SLSQP',options={'maxiter': maxiter, 'disp': True})
print(res)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.51047874449416
            Iterations: 6
            Function evaluations: 8
            Gradient evaluations: 6
     fun: 0.51047874449416
     jac: array([1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 8
     nit: 6
    njev: 6
  status: 0
 success: True
       x: array([0.24979695, 0.26068179])


In [9]:
#case of identically distriubted variables with rho = 0.9 and alpha = 1

M = 100000
rho = [[1, 0.9],[0.9, 1]]
lam = [3, 3]
T = 1
mu = 0
sigma = 1
obj = CompoundPoisson(T, lam, rho,  M, mu, sigma)
distr = obj.compound_poisson()

c = 1.
alpha  = 1.
loss = MCLossFunction(distr, alpha, c)
maxiter = 3500

init = np.ones(loss.dim)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})
res = minimize(loss.objective, init, jac = loss.objective_jac, constraints = cons, method='SLSQP',options={'maxiter': maxiter, 'disp': True})
print(res)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.5208186402727395
            Iterations: 5
            Function evaluations: 6
            Gradient evaluations: 5
     fun: 0.5208186402727395
     jac: array([1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 6
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([0.25963206, 0.26118658])


In [5]:
M = 100000
rho = [[1, 0.9],[0.9, 1]]
lam = [100, 100]
T = 1
mu = 0
sigma = 1
obj = CompoundPoisson(T, lam, rho,  M, mu, sigma)
distr = obj.compound_poisson()

c = 1.
alpha  = 1.
loss = MCLossFunction(distr, alpha, c)
maxiter = 3500

init = np.ones(loss.dim)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})
res = minimize(loss.objective, init, jac = loss.objective_jac, constraints = cons, method='SLSQP',options={'maxiter': maxiter, 'disp': True})
print(res)

ValueError: the input matrix must be positive semidefinite